# Geospatial Modeling: Road Network & Camera FOV
This notebook pulls road network data from OpenStreetMap using OSMnx, creates GeoDataFrames for approach legs and camera FOV polygons, overlays traffic metrics, and visualizes with Folium.

In [ ]:
# Install required packages (uncomment if needed)
%pip install osmnx geopandas folium pandas

In [ ]:
import osmnx as ox
import geopandas as gpd
import folium
import pandas as pd
from shapely.geometry import Polygon, Point

## 1. Download Road Network from OSM
Specify the location and download the road network graph.

In [ ]:
# Example: Rochester, NY
place = 'Rochester, New York, USA'
G = ox.graph_from_place(place, network_type='drive')
edges = ox.graph_to_gdfs(G, nodes=False)
edges.head()

## 2. Camera FOV & Approach Legs
Create polygons for camera fields of view and approach legs.

In [ ]:
# Camera FOV and approach polygons from lane_config
camera_fovs = [
    {
        "camera_id": "Skyline-5557",
        "geometry": Polygon([(0,110),(140,110),(140,140),(0,140)])  # monroe_winton_wb_thru_1
    },
    {
        "camera_id": "Skyline-5557",
        "geometry": Polygon([(0,140),(150,140),(150,165),(0,165)])  # monroe_winton_wb_thru_2
    },
    {
        "camera_id": "Skyline-5557",
        "geometry": Polygon([(210,105),(351,105),(351,140),(210,140)])  # monroe_winton_eb_thru_1
    },
    {
        "camera_id": "Skyline-5557",
        "geometry": Polygon([(200,140),(351,140),(351,170),(200,170)])  # monroe_winton_eb_thru_2
    },
    {
        "camera_id": "Skyline-5557",
        "geometry": Polygon([(150,40),(225,60),(205,120),(140,110)])  # monroe_winton_sb_approach
    },
    {
        "camera_id": "Skyline-5557",
        "geometry": Polygon([(270,185),(351,175),(351,239),(230,239)])  # monroe_winton_nb_right
    },
    {
        "camera_id": "Skyline-13982",
        "geometry": Polygon([(185,95),(230,95),(230,200),(185,200)])  # winton_elmwood_nb_right
    },
    {
        "camera_id": "Skyline-13982",
        "geometry": Polygon([(230,90),(275,90),(275,200),(230,200)])  # winton_elmwood_nb_middle
    },
    {
        "camera_id": "Skyline-13982",
        "geometry": Polygon([(275,85),(330,85),(330,200),(275,200)])  # winton_elmwood_nb_left
    }
]

fov_gdf = gpd.GeoDataFrame(camera_fovs)
fov_gdf.head()

## 3. Overlay Metrics
Load metrics (CSV or from Kafka) and join with camera polygons.

In [ ]:
# Install kafka-python if needed
# %pip install kafka-python

import json
from kafka import KafkaConsumer
import pandas as pd

# Connect to Kafka and consume metrics
consumer = KafkaConsumer(
    'metrics.minute',
    bootstrap_servers='localhost:9092',
    auto_offset_reset='latest',
    value_deserializer=lambda m: json.loads(m.decode('utf-8')),
    consumer_timeout_ms=5000  # Stop after 5 seconds if no new messages
)

metrics = []
for msg in consumer:
    metrics.append(msg.value)

consumer.close()

# Flatten and load into DataFrame
if metrics:
    # Each message contains a 'minute' and 'metrics' dict
    rows = []
    for m in metrics:
        minute = m.get('minute')
        for lane_id, lane_metrics in m.get('metrics', {}).items():
            row = {'minute': minute, 'lane_id': lane_id}
            row.update(lane_metrics)
            rows.append(row)
    metrics_df = pd.DataFrame(rows)
    display(metrics_df.head())

    # --- Merge metrics with camera polygons ---
    # Assume lane_id matches camera_id or can be mapped (adjust as needed)
    # For demo, join on camera_id if present in metrics_df
    if 'camera_id' in metrics_df.columns:
        merged_gdf = fov_gdf.merge(metrics_df, left_on='camera_id', right_on='camera_id', how='left')
    else:
        # If no camera_id, just preview metrics_df and fov_gdf
        merged_gdf = fov_gdf.copy()
        merged_gdf = merged_gdf.join(metrics_df, how='left')

    display(merged_gdf.head())
else:
    print("No metrics found in Kafka topic.")

## 4. Visualize with Folium
Display camera FOVs and overlay metrics as heatmap or colored polygons.

In [ ]:
# Print normalized average KPI metrics for each intersection (camera_id), excluding zeros
if 'metrics_df' in locals():
    print("--- Normalized KPI Metrics Averages by Intersection (camera_id) ---")
    kpi_cols = [col for col in ['queue_length', 'avg_headway', 'avg_speed'] if col in metrics_df.columns]
    # Exclude zeros for avg_headway and avg_speed
    def nonzero_mean(series):
        nz = series[series > 0]
        return nz.mean() if len(nz) > 0 else 0
    if kpi_cols:
        if 'camera_id' in metrics_df.columns:
            avg_df = metrics_df.groupby('camera_id').agg({
                'queue_length': 'mean',
                'avg_headway': nonzero_mean,
                'avg_speed': nonzero_mean
            }).reset_index()
            # Normalize avg_speed and avg_headway columns (min-max normalization)
            for col in ['avg_speed', 'avg_headway']:
                if col in avg_df.columns:
                    min_val = avg_df[col].min()
                    max_val = avg_df[col].max()
                    if max_val > min_val:
                        avg_df[col + '_norm'] = (avg_df[col] - min_val) / (max_val - min_val)
                    else:
                        avg_df[col + '_norm'] = 0
            print(avg_df)
        else:
            print("camera_id column not found. Showing overall averages:")
            avg_vals = {}
            for col in kpi_cols:
                avg_vals[col] = nonzero_mean(metrics_df[col])
            # Normalize
            for col in ['avg_speed', 'avg_headway']:
                if col in avg_vals:
                    min_val = metrics_df[col][metrics_df[col] > 0].min()
                    max_val = metrics_df[col][metrics_df[col] > 0].max()
                    val = avg_vals[col]
                    if max_val > min_val and val > 0:
                        avg_vals[col + '_norm'] = (val - min_val) / (max_val - min_val)
                    else:
                        avg_vals[col + '_norm'] = 0
            print(avg_vals)
        print("\nAvailable columns:", metrics_df.columns.tolist())
    else:
        print("No KPI columns found in metrics_df.")
else:
    print("metrics_df not found.")

In [ ]:
# Folium map visualization of camera polygons and KPIs
intersection_coords = [
    [43.12708222627869, -77.5653911674301],  # Monroe Ave & S Winton Rd
    [43.111108, -77.547803]                  # Monroe Ave & Clover St (NY65)
 ]
center_lat = sum([lat for lat, lon in intersection_coords]) / len(intersection_coords)
center_lon = sum([lon for lat, lon in intersection_coords]) / len(intersection_coords)

m = folium.Map(location=[center_lat, center_lon], zoom_start=15, tiles="OpenStreetMap")

intersection_names = [
    'Monroe Ave & S Winton Rd',
    'Monroe Ave & Clover St (NY65)'
 ]
for i, (lat, lon) in enumerate(intersection_coords):
    folium.Marker([lat, lon], popup=intersection_names[i]).add_to(m)

if 'merged_gdf' in locals():
    kpi_col = None
    for col in ['queue_length', 'volume', 'avg_headway', 'avg_speed']:
        if col in merged_gdf.columns:
            kpi_col = col
            break
    if kpi_col:
        min_kpi = merged_gdf[kpi_col].min()
        max_kpi = merged_gdf[kpi_col].max()
        def get_color(val):
            if pd.isnull(val): return '#cccccc'
            ratio = (val - min_kpi) / (max_kpi - min_kpi) if max_kpi > min_kpi else 0
            r = int(255 * ratio)
            g = int(255 * (1 - ratio))
            return f'#{r:02x}{g:02x}20'
        for _, row in merged_gdf.iterrows():
            color = get_color(row[kpi_col])
            popup_text = f"Camera: {row['camera_id']}<br>{kpi_col}: {row[kpi_col]}"
            folium.GeoJson(
                row['geometry'],
                name=row['camera_id'],
                style_function=lambda x, color=color: {'fillColor': color, 'color': color, 'weight': 2, 'fillOpacity': 0.6},
                tooltip=popup_text
            ).add_to(m)
    else:
        for _, row in merged_gdf.iterrows():
            folium.GeoJson(row['geometry'], name=row['camera_id']).add_to(m)
else:
    for _, row in fov_gdf.iterrows():
        folium.GeoJson(row['geometry'], name=row['camera_id']).add_to(m)

m

In [ ]:
m.save("test_map.html")

In [ ]:
# Visualize metrics and polygons using kepler.gl
# %pip install keplergl
from keplergl import KeplerGl
from IPython.display import display

# Use merged_gdf if available, else fallback to fov_gdf
if 'merged_gdf' in locals():
    kepler_map = KeplerGl(height=600)
    kepler_map.add_data(data=merged_gdf, name="Camera Polygons & Metrics")
    display(kepler_map)
else:
    kepler_map = KeplerGl(height=600)
    kepler_map.add_data(data=fov_gdf, name="Camera Polygons")
    display(kepler_map)